The goal here is to find one non-repeting pattern in an image. First, find the pattern, then second, compute correlation.

It is based on the post https://laurentperrinet.github.io/sciblog/posts/2021-12-01-find-a-8-in-a-forest-of-9.html

Let's first initialize the notebook:

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import matplotlib.pyplot as plt
import numpy as np
phi = (np.sqrt(5)+1)/2
fig_width = 10
figsize = (fig_width, fig_width/phi)

In [ ]:
import imageio as io
im = io.v2.imread('files/2021-12-01-99999999999998.jpg')
im = im.sum(axis=-1)

In [ ]:
N_X, N_Y = im.shape
N_X, N_Y

Original image

In [ ]:
fig, ax = plt.subplots(figsize=(fig_width, fig_width))
ax.matshow(im, cmap=plt.gray());

Zoom

In [ ]:
fig, ax = plt.subplots(figsize=(fig_width, fig_width))
ax.matshow(im[:40, :40]);

In [ ]:
fig, ax = plt.subplots(figsize=(fig_width, fig_width))
ax.matshow(im[-40:, -40:]);

Crop

In [ ]:
im = im[:, 24:1194]
fig, ax = plt.subplots(figsize=(fig_width, fig_width))
ax.matshow(im, cmap=plt.gray());

Average on one axis

In [ ]:
im_x = im.mean(axis=1)
im_x = np.roll(im_x, N_X//2) # avoid border effects
#im_x = np.correlate(im_x, [-1, 2, 1], 'same')*1. # contrast detection
im_x -= im_x.mean()
fig, ax = plt.subplots(figsize=(fig_width, fig_width/phi**2))
ax.plot(im_x[:100]);

Cross-correlation

In [ ]:
xcorr = np.correlate(im_x, im_x, 'same')*1.
xcorr /= xcorr.max()
fig, ax = plt.subplots(figsize=(fig_width, fig_width/phi**2))
ax.plot(xcorr);

In [ ]:
N_X//2, im_x.shape, xcorr.shape, im.shape

In [ ]:
fig, ax = plt.subplots(figsize=(fig_width, fig_width/phi**2))
ax.plot(xcorr[(N_X//2+1):(50+N_X//2)]);

In [ ]:
period_X = np.argmax(xcorr[(1+N_X//2):]) + 1

In [ ]:
print(f'{period_X=}')

The other dimension:

In [ ]:
im_y = im.mean(axis=0)
im_y -= im_y.mean()
#im_y[-1] = im_y[0]
#im_y[-2] = im_y[1]
im_y = np.roll(im_y, N_Y//2) # avoid border effects
#im_y = np.correlate(im_y, [-1, 2, -1], 'same')*1. # contrast detection

In [ ]:
fig, ax = plt.subplots(figsize=(fig_width, fig_width/phi**2))
ax.plot(im_y);

In [ ]:
fig, ax = plt.subplots(figsize=(fig_width, fig_width/phi**2))
ax.plot(im_y[:40]);

In [ ]:
xcorr = np.correlate(im_y, im_y, 'same')*1.

In [ ]:
xcorr /= xcorr.max()
fig, ax = plt.subplots(figsize=(fig_width, fig_width/phi**2))
ax.plot(xcorr[(N_Y//2+3):]);

In [ ]:
xcorr /= xcorr.max()
fig, ax = plt.subplots(figsize=(fig_width, fig_width/phi**2))
ax.plot(xcorr[(N_Y//2+3):(40+N_Y//2)]);

In [ ]:
period_Y = np.argmax(xcorr[(3+N_Y//2):]) + 3

In [ ]:
print(f'{period_Y=}')

One sample (other random choices would *most probably* fit - do not go in the borders!)

In [ ]:
idx, idy = 0, 26
kernel = im[idx:idx+period_X, idy:idy+period_Y]

In [ ]:
fig, ax = plt.subplots(figsize=(fig_width, fig_width))
ax.matshow(kernel);

compute cross correlation using 
https://laurentperrinet.github.io//sciblog/posts/2017-09-20-the-fastest-2d-convolution-in-the-world.html


In [ ]:
from numpy.fft  import fft2, ifft2
xcorr = np.real(ifft2(fft2(im)*fft2(kernel, s=im.shape)))

Now you can spot where there is a difference (and then in the rest due to kerning...)

In [ ]:
fig, ax = plt.subplots(figsize=(fig_width, fig_width))
ax.matshow(xcorr, cmap=plt.magma());

## some book keeping for the notebook

In [ ]:
%load_ext watermark
%watermark -i -h -m -v -p numpy,matplotlib,imageio  -r -g -b